In [1]:
import autokeras
import pandas
import numpy
import os
from PIL import Image

In [2]:
def load_dataset(dirname, label):
    X = []
    Y = []
    filenames = os.listdir(dirname)
    for filename in filenames:
        img = Image.open("{}/{}".format(dirname, filename))
        arr = numpy.array(img)
        X.append(arr)
        Y.append(label)
    return X, Y

In [3]:
X_0, Y_0 = load_dataset("data/0", 0)
X_1, Y_1 = load_dataset("data/1", 1)

In [4]:
train_rate = 0.5
train_X_0 = X_0[0:int(len(X_0)*train_rate)]
train_X_1 = X_1[0:int(len(X_1)*train_rate)]
train_Y_0 = Y_0[0:int(len(Y_0)*train_rate)]
train_Y_1 = Y_1[0:int(len(Y_1)*train_rate)]

test_X_0 = X_0[int(len(X_0)*train_rate):]
test_X_1 = X_1[int(len(X_1)*train_rate):]
test_Y_0 = Y_0[int(len(Y_0)*train_rate):]
test_Y_1 = Y_1[int(len(Y_1)*train_rate):]

train_X = numpy.r_[train_X_0, train_X_1]
train_Y = numpy.r_[train_Y_0, train_Y_1]
test_X = numpy.r_[test_X_0, test_X_1]
test_Y = numpy.r_[test_Y_0, test_Y_1]

In [5]:
train_Y[0:2]

array([0, 0])

In [6]:
train_X[0:2]

array([[[3, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 2, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]], dtype=uint8)

In [7]:
model = autokeras.ImageClassifier()

In [8]:
print(len(train_X), len(train_Y))
print(train_X.shape)
print(train_Y.shape)

923 923
(923, 28, 28)
(923,)


In [9]:
model.fit(train_X, train_Y, time_limit=0.2*60*60)

In [10]:
model.final_fit(
    train_X,
    train_Y,
    train_X,
    train_Y,
    retrain=False)

In [11]:
model.predict(test_X[0:3])

array([0, 0, 0])

In [12]:
# save
path = "model.hdf5"
model.export_autokeras_model(path)

In [13]:
output_model = autokeras.utils.pickle_from_file(path)

In [14]:
output_model.graph.layer_list.pop(-1)

In [15]:
vec_train_X = output_model.predict(train_X)
vec_test_X = output_model.predict(test_X)

In [16]:
pandas.DataFrame(vec_train_X).head(2)

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,5.377497,2.031294,0.0,0.000000,1.709647,0.196155,0.271180,0.658500,1.523646,0.797557,...,0.0,1.456105,0.0,2.202573,0.0,0.0,0.745896,1.844144,0.0,0.0
1,2.497815,0.804688,0.0,0.029483,0.843897,0.117921,0.124081,0.337379,0.705135,0.263632,...,0.0,0.618370,0.0,0.927023,0.0,0.0,0.430092,0.765334,0.0,0.0


In [17]:
from sklearn.linear_model import LogisticRegression

In [18]:
lr = LogisticRegression(solver="lbfgs")

In [19]:
lr.fit(vec_test_X, test_Y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [20]:
lr.score(vec_test_X, test_Y)

0.9945945945945946